# Run clustering code on Breakthrough network


In [ ]:
import pypsa

import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
import geopy.distance
import numpy as np

from collections import namedtuple

In [ ]:
%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append('scripts/')

import simplify_network as simplify_network
import cluster_network as cluster_network

In [ ]:
region = "Eastern"

In [ ]:
n = pypsa.Network(f"../../singularity/carbon-flow/data/opf_ready_networks/{region}_breakthrough.nc")

In [ ]:
def check_n(n):
    gascap = n.generators[n.generators.carrier != 'load'].groupby('carrier').p_nom.sum()['ng']
    assert gascap == 383819.7


In [ ]:
check_n(n)

In [ ]:
n.lines.v_nom.unique()

In [ ]:
n.lines.type.value_counts()

In [ ]:
n, map = simplify_network.simplify_network_to_380(n, to_line_type="Breakthrough_230.0_lowr")

In [ ]:
check_n(n)

In [ ]:
out_config = namedtuple('output', ["connection_costs"])("egcost.csv")


In [ ]:
n.generators["weight"] = np.nan

In [ ]:
# Remove generators with zero p_nom
n.mremove("Generator",n.generators[n.generators.p_nom==0].index)

In [ ]:
check_n(n)

In [ ]:
def single_mode(s:pd.Series): 
    modes= s.mode()
    if len(modes)>0:
        return modes.loc[0]
    return None

# How should we aggregate each numeric column in generators? 
aggregation_strategies = {
    "generators":{
        "p_set":pd.Series.median,
        "q_set":pd.Series.median,
        "marginal_cost":pd.Series.median, 
        'q_max':pd.Series.median, 
        'q_min':pd.Series.median, 
        'v_set_pu':pd.Series.median, 
        'mva_base':pd.Series.median, 
       'p_min':pd.Series.max, 
       'Pc1':pd.Series.median, 
       'Pc2':pd.Series.median, 
       'Qc1min':pd.Series.median, 
       'Qc1max':pd.Series.median, 
       'Qc2min':pd.Series.median, 
       'Qc2max':pd.Series.median,
       'ramp_agc':pd.Series.median, 
       'ramp_10':pd.Series.median, 
       'ramp_30':pd.Series.median, 
       'ramp_q':pd.Series.median, 
       'apf':pd.Series.median, 
       'marginal_cost':pd.Series.median,
       'p_nom_min':pd.Series.sum, 
       'p_nom_max':pd.Series.sum, 
       'p_min_pu':pd.Series.median, 
       'p_max_pu':pd.Series.median, 
       'sign':pd.Series.mode, 
       'build_year':pd.Series.median,
       'lifetime':pd.Series.median, 
       'capital_cost':pd.Series.sum, 
       'efficiency':pd.Series.median, 
       'start_up_cost':pd.Series.sum,
       'shut_down_cost':pd.Series.sum, 
       'min_up_time':pd.Series.median, 
       'min_down_time':pd.Series.median, 
       'up_time_before':pd.Series.median,
       'down_time_before':pd.Series.median, 
       'ramp_limit_up':pd.Series.max, 
       'ramp_limit_down':pd.Series.max,
       'ramp_limit_start_up':pd.Series.median, 
       'ramp_limit_shut_down':pd.Series.median, 
       'p_nom_opt':pd.Series.median
    },
    "buses":{
        'Pd':pd.Series.median, 
        'Qd':pd.Series.median,
        'Gs':pd.Series.median,
        'Bs':pd.Series.median,
        'area':pd.Series.median,
        'v_mag_pu_set':pd.Series.median,
        'v_ang_set':pd.Series.median,
        'v_nom':pd.Series.median,
        'zone':single_mode, 
        'v_mag_pu_max':pd.Series.median,
        'v_mag_pu_min':pd.Series.median,
        'control':single_mode, 
        'x':pd.Series.median,
        'y':pd.Series.median,
        'type':single_mode,
        'carrier':single_mode,
        'unit':single_mode,
        'sub_network':single_mode,
        'country':single_mode,
        'generator':single_mode,
    }
}

In [ ]:
# Find subnetworks
n.determine_network_topology()

In [ ]:
# All subnetworks but 1 are unconnected. Remove them. 
# These aren't removed by remove_stubs because the're not stubs, they're just unconnected buses. 
n.buses.groupby('sub_network').count()['Pd'].unique()

In [ ]:
check_n(n)

In [ ]:
# How much generation capacity is connected to buses we're about to remove? 
buslist = n.buses[n.buses.sub_network != '0'].index
print(f"unconnected cap {n.generators[n.generators.bus.isin(buslist)].p_nom.sum()}")
print(f"connected cap {n.generators[~n.generators.bus.isin(buslist)].p_nom.sum()}")

In [ ]:
# Get some info. Where are these? what carriers? what sorts of caps? 
disconnected_gens = n.generators[n.generators.bus.isin(buslist)][["bus", "p_nom","carrier","marginal_cost"]]
disconnected_gens = disconnected_gens.merge(n.buses[["x","y"]], how='left', left_on='bus', right_index=True)
#px.scatter_geo(disconnected_gens, lat='y', lon='x', color='carrier')

In [ ]:
def get_nearest_bus(buses: pd.DataFrame, loc: tuple):
    """
    Return bus nearest to loc (lat, long tuple)
    """
    distances = buses.apply(
        lambda b: geopy.distance.geodesic(loc, (b.y, b.x)).km,
        axis=1,
    )
    return distances.idxmin()

In [ ]:
# Connect disconnected generators to the nearest connected bus 
avail_buses = n.buses[n.buses.sub_network=='0']
for gen in disconnected_gens.index:
    bus_loc = n.buses.loc[n.generators.loc[gen,'bus'], ["x","y"]]
    same_loc = avail_buses.loc[(avail_buses.x==bus_loc.x) & (avail_buses.y==bus_loc.y)].index
    if len(same_loc) >0:
        assign_bus = same_loc[0]
    else:
        print(f"Warning: no buses at location of generator {gen}, using nearest bus.")
        assign_bus = get_nearest_bus(avail_buses, bus_loc)
    n.generators.loc[gen,'bus'] = assign_bus

In [ ]:
# Now how much generation capacity is connected to buses we're about to remove? 
buslist = n.buses[n.buses.sub_network != '0'].index
print(f"unconnected cap {n.generators[n.generators.bus.isin(buslist)].p_nom.sum()}")
print(f"connected cap {n.generators[~n.generators.bus.isin(buslist)].p_nom.sum()}")

In [ ]:
check_n(n)

In [ ]:
n.mremove("Bus", n.buses[n.buses.sub_network != '0'].index)

In [ ]:
n.buses.sub_network.unique()

In [ ]:
check_n(n)

In [ ]:
config = {"clustering":
        {"simplify_network":{
        "to_substations": False, # network is simplified to nodes with positive or negative power injection (i.e. substations or offwind connections)
        "algorithm": "kmeans", # choose from: [hac, kmeans]
        "exclude_carriers": []
    }}}

n, stub_map = simplify_network.remove_stubs(n, {}, config, out_config, aggregation_strategies)

In [ ]:
exclude_carriers = []
aggregate_carriers = set(n.generators.carrier) - set(exclude_carriers)

In [ ]:
n.buses["country"]="USA"

In [ ]:
# busmap for 
n.calculate_dependent_values()
len(n.buses.sub_network.unique())

In [ ]:
check_n(n)

In [ ]:
n.export_to_netcdf("networks/Eastern_breakthrough_s.nc")

In [ ]:
#N = int(len(n.buses)/5)
#print(N)

In [ ]:
clustering = cluster_network.clustering_for_n_clusters(
    n, 
    1024, # Note: this is much more clustered than pyPSA-Eur
    False, # custom busmap?
    #n.buses.sub_network, # when clusters = n subnets  
    aggregate_carriers,
    1.25, # default from config.yaml
    aggregation_strategies,
    "ipopt",
    "kmeans",
    "", # shouldn't matter
    40.0, # HVAC overhead cost. not sure if correct but shouldn't matter since we're not cap expanding
    None 
    )

In [ ]:
check_n(n)

In [ ]:
cluster_network.update_p_nom_max(clustering.network)

In [ ]:
check_n(n)

In [ ]:
clustering.network.generators.p_nom.sum()

In [ ]:
clustering.network.plot()

In [ ]:
n_clustered = clustering.network.copy()
n_clustered.line_types = n.line_types
n_clustered.consistency_check()
n_clustered.export_to_netcdf("networks/Eastern_breakthrough_s_1024.nc")
#clustering.network.lopf(solver_name='highs', snapshots=n.snapshots[0:24], pyomo=False)

In [ ]:
check_n(n_clustered)

In [ ]:
n_clustered.generators.marginal_cost.median()